In [ ]:
REPO_NAME = "BSides-Nova-BreakAI-Workshop-2025"
!git clone https://github.com/pavanreddyml/{REPO_NAME}.git
!mv {REPO_NAME}/* . && mv {REPO_NAME}/.* . 2>/dev/null
!rm -rf {REPO_NAME}
!pip install -e bsidesnova

!pip install -e adversarial-lab

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(restart=True)

In [ ]:
from adversarial_lab.analytics import *
from adversarial_lab.db import SqlliteDB
from adversarial_lab.callbacks import EarlyStopping
from adversarial_lab.core.optimizers import PGD
from adversarial_lab.core.penalties import LpNorm
from adversarial_lab.core.losses import CategoricalCrossEntropy
from adversarial_lab.attacker.adversarial import WhiteBoxMisclassificationAttack
from adversarial_lab.core.noise_generators.tensor import AdditiveNoiseGenerator
from adversarial_lab.core.preprocessing import PreprocessingFromFunction
from adversarial_lab.core.constraints import POClip, PONoisedSampleBounding
from adversarial_lab.arsenal.attacks_base import AttacksBase

from adversarial_lab.utils import VideoPlotter, Plotting

from PIL import Image
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

from bsidesnova import *
from bsidesnova.selectors import *
from bsidesnova.ui import *

In [ ]:
widget = AttackSelectorUI()
widget.display()

In [ ]:
selector = AdversarialSelector(root_path="")
conf = widget.get_configuration()
att_conf = selector.attacker_selector(conf)
att_conf["on_origial"] = True

In [ ]:
if conf["attack"] in ["Deep Fool", "Smooth Fool"]:
  early_stopping = EarlyStopping(trigger="misclassification")
else:
  early_stopping = EarlyStopping(trigger="misclassification", target_class=2, confidence=0.5)


In [ ]:
db = SqlliteDB(db_path="whitebox.db")
analytics = AdversarialAnalytics(
    db=db,
    trackers=[
        ImageTracker(),
        LossTracker(),
        NoiseStatisticsTracker(),
        NoiseTracker(),
        PredictionsTracker()
    ],
    table_name="adv",
    force_create_table=True

)

In [ ]:
attacker_class = att_conf["attacker"]
model = att_conf["model"]
preprocess = att_conf["preprocess"]
attack_params = att_conf["attack_params"]
input_shape = att_conf["input_shape"]
sample = att_conf["sample"]
decode_preds = att_conf["decode_preds"]
target_class = att_conf["target_class"]

attack_params["analytics"] = analytics
attack_params["callbacks"] = [early_stopping]

if conf["category"] == 'whitebox':
  def predict(sample):
      return model.predict(preprocess(sample), verbose=0)
elif conf["category"] == 'blackbox':
  def predict(sample):
    sample = np.array([sample])
    preds = model.predict(sample, verbose=0)
    return [pred for pred in preds]

predictions = predict(sample)

In [ ]:
attacker: AttacksBase = attacker_class(model=model, preprocessing_fn=PreprocessingFromFunction.create(preprocess), **attack_params)
noise, noise_meta = attacker.attack(sample=sample, target_class=60, epochs=20)

In [ ]:
plotter = Plotting(db=db, table_name="adv")
plotter.plot_losses()
plotter.plot_predictions(n=5)
plotter.plot_noise_statistics("mean")
plotter.plot_samples(
    image=sample,
        noise=noise,
        config=None,
        class_dict=None,
        original_class_idx=None,
        predicted_class_idx=None,
        original_confidence=None,
        predicted_confidence=None)

In [ ]:
plotter = VideoPlotter(db=db, table_name="adv")
plotter.make_video()

In [ ]:
# display the generated adversarial video output_video.mp4
from IPython.display import Video
Video("output_video.mp4")

In [ ]:
from adversarial_lab.defenses.preprocessing import *

In [ ]:
# Preprocessing UI (Checkboxes and Randomize, max_num)
# Preprocessing Selector
#